In [1]:
import numpy as np
import pandas as pd
import gc
import time
import lightgbm as lgb
import matplotlib.pyplot as plt
import pickle

In [2]:
df = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/FEATURE_AGG_2012TO2019.pkl")

In [3]:
[col for col in df.columns if "LAST_" not in col]

['GAME_ID',
 'TEAM_ID_HOME',
 'TEAM_ID_AWAY',
 'PLUS_MINUS',
 'GAME_DATE',
 'TEAM_ID_H',
 'TEAM_ID_A']

In [4]:
len([col for col in df.columns if "_H" in col])

362

In [5]:
len([col for col in df.columns if ("_A" in col) & ("_H" not in col)])

362

In [6]:
set(df.columns) - set([col for col in df.columns if "_H" in col]) - set([col for col in df.columns if ("_A" in col) & ("_H" not in col)])

{'GAME_DATE', 'GAME_ID', 'PLUS_MINUS'}

In [7]:
df["TEAM_ID_AWAY"] = df["TEAM_ID_AWAY"].astype("category")
df["TEAM_ID_HOME"] = df["TEAM_ID_HOME"].astype("category")

In [8]:
column = ["_".join(col.split("_")[2:-1]) for col in df.columns if ("LAST_1_" in col)&("_H" in col)]

In [9]:
df.loc[df[df.loc[:,"PLUS_MINUS"]>0].index,"PLUS_MINUS"] = 1
df.loc[df[df.loc[:,"PLUS_MINUS"]<0].index,"PLUS_MINUS"] = 0

In [10]:
new_col = []
for col_i in column:
    new_col += [col for col in df.columns if col_i in col]

In [11]:
new_col = list(set(df.columns)-set(new_col))+list(new_col)

In [12]:
len(set(new_col))

727

In [13]:
df = df[new_col]

In [14]:
use_col = pickle.load(open("/Users/chienan/Pycon/github/SportLottery/model/classfication_feature-45_columns.pkl","rb"))

In [15]:
x_train = df.loc[:, use_col]
y_train = df.loc[:, "PLUS_MINUS"]

In [16]:
train_dataset = lgb.Dataset(x_train, y_train)

In [17]:
best_params = pickle.load( open("/Users/chienan/Pycon/github/SportLottery/model/model_feature-45_params.pkl","rb"))

In [18]:
best_params.update({'num_iterations': 93, 
                    "metric": "binary_error",
                    "objective": "binary"})
evals_result={}

In [19]:
best_params

{'bagging_fraction': 0.8843237532597547,
 'early_stopping_round': 100,
 'feature_fraction': 0.3511957417701298,
 'lambda_l1': 2.86312666321977,
 'lambda_l2': 0.8281471449920853,
 'learning_rate': 0.04583145032209364,
 'max_depth': 6,
 'metric': 'binary_error',
 'min_child_weight': 34.58297582508912,
 'min_split_gain': 0.03766475721581549,
 'num_iterations': 93,
 'num_leaves': 34,
 'objective': 'binary'}

In [20]:
%%time 
print("start training...")
model = lgb.train(train_set=train_dataset,
                  valid_sets=[train_dataset] ,
                  verbose_eval=20,
                  params=best_params,
                  evals_result=evals_result)

start training...
Training until validation scores don't improve for 100 rounds.


/Users/chienan/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/chienan/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:107: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[20]	training's binary_error: 0.323549
[40]	training's binary_error: 0.302438
[60]	training's binary_error: 0.288666
[80]	training's binary_error: 0.277263
Did not meet early stopping. Best iteration is:
[92]	training's binary_error: 0.272522
CPU times: user 876 ms, sys: 690 ms, total: 1.57 s
Wall time: 645 ms


In [21]:
model.save_model('/Users/chienan/Pycon/github/SportLottery/model/classfication_model_feature-45.txt')